In [4]:
# Import necessary libraries
import numpy as np
import pandas as pd
import os
import json
from skimage.exposure import adjust_gamma
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Convolution2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from scipy import ndimage
from scipy.misc import imresize


# Get steering angles for controlled driving
angles = pd.read_csv('./data/driving_log.csv', header=None)
angles.columns = ('Center Image','Left Image','Right Image','Steering Angle','Throttle','Brake','Speed')
angles = np.array(angles['Steering Angle'])

# Get steering angles for recovery driving
recovery_angles = pd.read_csv('./recovery/driving_log.csv', header = None)
recovery_angles.columns = ('Center Image','Left Image','Right Image','Steering Angle','Throttle','Brake','Speed')
recovery_angles = np.array(recovery_angles['Steering Angle'])

# Create arrays for center, right and left images of controlled driving
images = np.asarray(os.listdir("./data/IMG/"))
center = np.ndarray(shape=(len(angles), 18, 64, 3))
right = np.ndarray(shape=(len(angles), 18, 64, 3))
left = np.ndarray(shape=(len(angles), 18, 64, 3))

# Create controlled driving datasets
# Images are resized to 32x64 to increase training speeds
# The top 12 pixels and bottom 2 pixels are cropped off because they contain irrelevant information for training behavior
# The final image size to be used in training is 18 x 64 x 3
count = 0
angles_num = len(angles)
for image in images:
    image_file = os.path.join('./data/IMG', image)
    if image.startswith('center'):
        image_data = ndimage.imread(image_file).astype(np.float32)
        center[count % angles_num] = imresize(image_data, (32,64,3))[12:30,:,:]
    elif image.startswith('right'):
        image_data = ndimage.imread(image_file).astype(np.float32)
        right[count % angles_num] = imresize(image_data, (32,64,3))[12:30,:,:]
    elif image.startswith('left'):
        image_data = ndimage.imread(image_file).astype(np.float32)
        left[count % angles_num] = imresize(image_data, (32,64,3))[12:30,:,:]
    count += 1

# Create an array for recovery driving images
recovery_images = np.asarray(os.listdir("./recovery/IMG/"))
recovery = np.ndarray(shape=(len(recovery_angles), 18, 64, 3))

# Create recovery driving dataset
count = 0
image_num = len(recovery)
while count < image_num:
    image_file = os.path.join('./recovery/IMG', recovery_images[0])
    image_data = ndimage.imread(image_file).astype(np.float32)
    recovery[count] = imresize(image_data, (32,64,3))[12:30,:,:]
    count += 1

# Concatenate all arrays into combined training dataset and labels
X_train = np.concatenate((center, right, left, recovery), axis=0)
y_train = np.concatenate((angles, (angles - .08), (angles + .08), recovery_angles),axis=0)

# Adjust gamma in images to increase contrast
X_train = adjust_gamma(X_train)

# Create a mirror image of the images in the dataset to combat left turn bias
mirror = np.ndarray(shape=(X_train.shape))
count = 0
for i in range(len(X_train)):
    mirror[count] = np.fliplr(X_train[i])
    count += 1
mirror.shape

# Create mirror image labels
mirror_angles = y_train * -1

# Combine regular features/labels with mirror features/labels
X_train = np.concatenate((X_train, mirror), axis=0)
y_train = np.concatenate((y_train, mirror_angles),axis=0)

# Perform train/test split to a create validation dataset
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=.05)

# Establish model architecture
model = Sequential()
model.add(BatchNormalization(axis=1, input_shape=(18,64,3)))
model.add(Convolution2D(16, 3, 3, border_mode='valid', subsample=(2,2), activation='relu'))
model.add(Convolution2D(24, 3, 3, border_mode='valid', subsample=(1,2), activation='relu'))
model.add(Convolution2D(36, 3, 3, border_mode='valid', activation='relu'))
model.add(Convolution2D(48, 2, 2, border_mode='valid', activation='relu'))
model.add(Convolution2D(48, 2, 2, border_mode='valid', activation='relu'))
model.add(Flatten())
model.add(Dense(512))
model.add(Dropout(.5))
model.add(Activation('relu'))
model.add(Dense(10))
model.add(Activation('relu'))
model.add(Dense(1))
model.summary()



____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_2 (BatchNorma (None, 18, 64, 3)     72          batchnormalization_input_2[0][0] 
____________________________________________________________________________________________________
convolution2d_6 (Convolution2D)  (None, 8, 31, 16)     448         batchnormalization_2[0][0]       
____________________________________________________________________________________________________
convolution2d_7 (Convolution2D)  (None, 6, 15, 24)     3480        convolution2d_6[0][0]            
____________________________________________________________________________________________________
convolution2d_8 (Convolution2D)  (None, 4, 13, 36)     7812        convolution2d_7[0][0]            
___________________________________________________________________________________________

In [7]:
# Compile model with adam optimizer and learning rate of .0001
adam = Adam(lr=0.0001)
model.compile(loss='mse',
              optimizer=adam)

# Model will save the weights whenever validation loss improves
checkpoint = ModelCheckpoint(filepath = './checkpoints/chk.{epoch:02d}-{val_loss:.2f}.hdf5', verbose = 1, save_best_only=True, monitor='val_loss')

# Stop training when validation loss fails to decrease
callback = EarlyStopping(monitor='val_loss', patience=2, verbose=1)

# Train model for 20 epochs and a batch size of 64
model.fit(X_train,
        y_train,
        nb_epoch=20,
        verbose=1,
        batch_size=64,
        shuffle=True,
        validation_data=(X_val, y_val),
        callbacks=[checkpoint, callback])

json_string = model.to_json()
with open('model.json', 'w') as jsonfile:
    json.dump(json_string, jsonfile)
model.save('model.h5')    
print("Model Saved")

Train on 24616 samples, validate on 1296 samples
Epoch 1/20
24616/24616 [==============================] - 56s - loss: 0.0226 - val_loss: 0.0262
Epoch 2/20
24616/24616 [==============================] - 54s - loss: 0.0222 - val_loss: 0.0266
Epoch 3/20
24616/24616 [==============================] - 57s - loss: 0.0219 - val_loss: 0.0264
Epoch 4/20
24616/24616 [==============================] - 54s - loss: 0.0216 - val_loss: 0.0267
Epoch 00003: early stopping
Model Saved
